In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from dipy.core.sphere import disperse_charges, HemiSphere
from dipy.core.gradients import gradient_table
from dipy.sims.voxel import multi_tensor, multi_tensor_odf
from dipy.data import get_fnames, get_sphere
from dipy.viz import window, actor
from dipy.reconst.dsi import (DiffusionSpectrumDeconvModel,
                              DiffusionSpectrumModel)

In [2]:
#S_gt = pd.read_pickle("train_results/S_test.pkl")
#S_predicted = pd.read_pickle("train_results/S_predicted.pkl")
F_gt = pd.read_pickle("train_results/F_test.pkl")
F_predicted = pd.read_pickle("train_results/F_pred.pkl")
angle_pairs = pd.read_pickle("synthetic_data/angle_pairs.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'train_results/F_test.pkl'

In [ ]:
evals = np.tile([0.0015, 0.00039, 0.00039], (len(angle_pairs), 1))

In [ ]:
evals

In [35]:
indices_top10 = np.argpartition(F_predicted[17], -10)[-10:]

# Get the top 10 largest elements
print(F_predicted[17][indices_top10])
print(indices_top10)

[0.03188691 0.03769178 0.07573513 0.14735374 0.19656003 0.13161968
 0.07782318 0.04147438 0.04751938 0.06559239]
[128  69 167   4 131 175  55  59 133 152]


In [36]:
indices_top3_true = np.argpartition(F_gt[11], -3)[-3:]
# Get the top 3 largest elements
print(F_gt[11][indices_top3_true])
print(indices_top3_true)

[0.        0.7311289 0.2688711]
[179  50  37]


In [38]:
sphere = get_sphere('repulsion724').subdivide(1)

for i in range(15):
    
    random_number = random.randint(0, 10000)
    odf_gt = multi_tensor_odf(sphere.vertices, evals, angles=angle_pairs,
                              fractions=F_gt[random_number])
    
    odf_predicted = multi_tensor_odf(sphere.vertices, evals, angles=angle_pairs,
                              fractions=F_predicted[random_number])
    
    
    from dipy.viz import window, actor
    
    # Enables/disables interactive visualization
    interactive = False
    
    
    scene = window.Scene()
    
    # concatenate data as 4D array
    odfs = np.vstack((odf_gt, odf_predicted))[:, None, None]
    odf_actor = actor.odf_slicer(odfs, sphere=sphere, scale=0.5, colormap='plasma')
    
    odf_actor.display(y=0)
    odf_actor.RotateX(90)
    scene.add(odf_actor)
    window.record(scene, out_path='comparisons/dsid_comparison_' + str(random_number) + '.png', size=(300, 300))
    if interactive:
        window.show(scene)